In [0]:
!sudo apt-get install -y python-enchant

In [0]:
# content of requirements.txt:
# numpy
# pyenchant
# tqdm
# nltk
# gensim
!pip install -r requirements.txt

In [0]:
dataset_2018_filename = "/content/drive/My Drive/QA/processed-data-2018.json"
dataset_2019_filename = "/content/drive/My Drive/QA/processed-data-2019.json"

In [0]:
import json
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from tqdm import tqdm
# from dataloaders.load_and_parse import load_all
from jaccard import get_similarity_score as j
import enchant
import re
import random
import numpy as np
import pickle
d = enchant.Dict("en_US")

In [0]:
from collections import namedtuple
import pickle
Datum = namedtuple('Datum', 'ref cite offsets author is_test facet year')
Offsets = namedtuple('Offsets', 'marker cite ref')
Article = namedtuple('Article', 'id xml sentences sections')

with open(dataset_2018_filename, 'rb') as f:
   dataset = pickle.load(f)

In [0]:
# 2018 - data for main task
import pickle

d = enchant.Dict("en_US")

# DATA_ROOT = '../../data'

# dataset = load_all(DATA_ROOT)


meaningless_file = open("meaningless_sentences.txt", "w")
alphabet_file = open("alphabet_sentences.txt", "w")


bad_ratios = []
bad_count = 0
total_count = 0
bad_sentences = []

def filter_meaningless(ref_article, i, jargon):
    line = ref_article[i]
    # line.replace(";", " ").replace(",", " ")
    enc_line = line.encode('unicode_escape').decode()
    line = re.sub(r"\\u....", "", enc_line)
    # Table caption
    if re.search(r'Table [0-9]+: ', line):
        start_idx = line.index("Table ")
        line = line[start_idx:]
        return True, line

    # Figure caption
    elif re.search(r'Figure [0-9]+: ', line):
        start_idx = line.index("Figure ")
        line = line[start_idx:]
        return True, line

    words = np.array(nltk.word_tokenize(ref_article[i]))
    num_singles = np.array([len(word) < 2 for word in words])
    count = words[num_singles].shape[0]
    ratio = count / len(words)

    if ratio > 0.53:
        # ff.write(ref_article[i])
        alphabet_file.write(ref_article[i] + "\n")
        return False, ""

    is_valid = []
    check = words
    for word in words:
        if d.check(word) or word in jargon:
            is_valid.append(True)
        else:
            is_valid.append(False)
    is_valid = np.array(is_valid)
    check = np.array(check)
    count = check[is_valid].shape[0]
    ratio = count / check.shape[0]
    # print(ratio, line)
    if ratio <= 0.72:
        # print("Ignoring Meaningless Sentence with ratio:", ratio, line)
        meaningless_file.write(ref_article[i] +  "\n")
        return False, ""
    return True, line

def clean_citing_sentence(sentence):
    if re.search(r"\\u....", sentence):
        sentence = re.sub(r"\\u....", "", sentence)
    return sentence

def prepare(dataset, reference_context):
    result = []
    jargon = open("jargon.txt", 'r').readlines()
    jargon = [word[:-1] for word in jargon]
    for data in tqdm(dataset):
        ref_article = data.ref.sentences
        try:
            ref_title = ref_article[0]
        except:
            ref_title = "No title"
        ref_sections = data.ref.sections
        citing_article = data.cite.sentences
        offsets = data.offsets
        positives = offsets.ref
        for k,v in ref_article:
            is_meaningful, ref_article[k] = filter_meaningless(ref_article, k, jargon) 
            if not is_meaningful:
                count_bad += 1
            
        negatives = ref_article.keys() - set(positives)
        if len(negatives) < 3 or not citing_article:
            continue
        positive_ids = []
        for x in positives:
            positive_ids.extend(
                [i for i in range(max(1, x - reference_context), min(x + reference_context, len(ref_article) - 1) + 1)])
        complete_citing_sentence = " ".join([citing_article[c] for c in offsets.cite])
        complete_citing_sentence = clean_citing_sentence(complete_citing_sentence.encode('unicode_escape').decode())
        negatives = set(negatives) - set(positives)
        r = {}
        negs = []
        for i in negatives:
            is_meaningful, line = filter_meaningless(ref_article, i, jargon)
            if is_meaningful:
                r[i] = line
                negs.append(i)
        negatives = negs
        text_sim_negatives = sorted(negatives, key=lambda x: j(r[x], complete_citing_sentence), reverse=True)[:1]
        rand_negs = np.random.choice(list(set(negatives) - set(text_sim_negatives)), size=2)
        negatives = text_sim_negatives + list(rand_negs)

        result.extend([(" ".join([r[i] for i in range(max(1, x - reference_context),
                                                                min(x + reference_context, len(ref_article)-1) + 1)]),
                        complete_citing_sentence, 0) for x in negatives])
        try:
            result.extend([(" ".join([ref_article[i] for i in range(max(1, x - reference_context),
                                                                    min(x + reference_context, len(ref_article)-1) + 1)]),
                            complete_citing_sentence, 1) for x in positives])
        except:
            print("hello")
    return result

reference_context = 0
result = prepare(dataset, reference_context)
with open("prepared_data_2018.json", "w") as f:
    json.dump(result, f)

# with open("ratios_0.53.txt", "w") as f:
#     f.write("count: {}\n".format(bad_count))
#     f.write("count: {}\n".format(total_count))
#     for i, j in zip(bad_sentences, bad_ratios):
#         f.write("S: {}, \t R: {}\n".format(i, str(j)))
#


100%|██████████| 753/753 [01:20<00:00,  9.38it/s]


In [0]:
from collections import namedtuple
Datum = namedtuple('Datum', 'ref cite offsets author is_test facet year')
Offsets = namedtuple('Offsets', 'marker cite ref')
Article = namedtuple('Article', 'id xml sentences sections')

with open(dataset_2019_filename, 'rb') as f:
   dataset = pickle.load(f)

In [0]:
# nsp data for 2019 dataset
meaningless_file = open("meaningless_sentences_2019.txt", "w")
alphabet_file = open("alphabet_sentences_2019.txt", "w")


bad_ratios = []
bad_count = 0
total_count = 0
bad_sentences = []

def filter_meaningless(ref_article, i, jargon):
    line = ref_article[i]
    # line.replace(";", " ").replace(",", " ")
    enc_line = line.encode('unicode_escape').decode()
    line = re.sub(r"\\u....", "", enc_line)
    # Table caption
    if re.search(r'Table [0-9]+: ', line):
        start_idx = line.index("Table ")
        line = line[start_idx:]
        return True, line

    # Figure caption
    elif re.search(r'Figure [0-9]+: ', line):
        start_idx = line.index("Figure ")
        line = line[start_idx:]
        return True, line

    words = np.array(nltk.word_tokenize(ref_article[i]))
    if len(words) == 0:
      return False, None
    num_singles = np.array([len(word) < 2 for word in words])
    count = words[num_singles].shape[0]
    ratio = count / len(words)

    if ratio > 0.53:
        # ff.write(ref_article[i])
        alphabet_file.write(ref_article[i] + "\n")
        return False, None

    is_valid = []
    check = words
    for word in words:
        if d.check(word) or word in jargon:
            is_valid.append(True)
        else:
            is_valid.append(False)
    is_valid = np.array(is_valid)
    check = np.array(check)
    count = check[is_valid].shape[0]
    ratio = count / check.shape[0]
    # print(ratio, line)
    if ratio <= 0.72:
        # print("Ignoring Meaningless Sentence with ratio:", ratio, line)
        meaningless_file.write(ref_article[i] +  "\n")
        return False, None
    return True, line

def clean_citing_sentence(sentence):
    if re.search(r"\\u....", sentence):
        sentence = re.sub(r"\\u....", "", sentence)
    return sentence

def prepare_nsp(dataset, reference_context):
    result = []
    jargon = open("jargon.txt", 'r').readlines()
    jargon = [word[:-1] for word in jargon]
    for data in tqdm(dataset):
        ref_article = data.ref.sentences
        try:
            ref_title = ref_article[0]
        except:
            ref_title = "No title"
        ref_sections = data.ref.sections
        citing_article = data.cite.sentences
        offsets = data.offsets
        positives = offsets.ref
        negatives = ref_article.keys() - set(positives)
        if len(negatives) < 3 or not citing_article:
            continue
        positive_ids = []
        for x in positives:
            positive_ids.extend(
                [i for i in range(max(1, x - reference_context), min(x + reference_context, len(ref_article) - 1) + 1)])
        complete_citing_sentence = " ".join([citing_article[c] for c in offsets.cite])
        complete_citing_sentence = clean_citing_sentence(complete_citing_sentence.encode('unicode_escape').decode())
        negatives = set(negatives) - set(positives)
        r = {}
        negs = []
        for i in negatives:
            is_meaningful, line = filter_meaningless(ref_article, i, jargon)
            if is_meaningful:
                r[i] = line
                negs.append(i)
        negatives = negs
        text_sim_negatives = sorted(negatives, key=lambda x: j(r[x], complete_citing_sentence), reverse=True)[:1]
        rand_negs = np.random.choice(list(set(negatives) - set(text_sim_negatives)), size=2)
        negatives = text_sim_negatives + list(rand_negs)

        result.extend([(" ".join([r[i] for i in range(max(1, x - reference_context),
                                                                min(x + reference_context, len(ref_article)-1) + 1)]),
                        complete_citing_sentence, 0) for x in negatives])
        try:
            result.extend([(" ".join([ref_article[i] for i in range(max(1, x - reference_context),
                                                                    min(x + reference_context, len(ref_article)-1) + 1)]),
                            complete_citing_sentence, 1) for x in positives])
        except:
            print("hello")
    return result

In [0]:
result = prepare_nsp(dataset, reference_context=0)
with open("prepared_data_nsp_main_2019.txt", "w") as f1:
    for sentence in result:
        f1.write(sentence + "\n")

In [0]:
result

In [0]:
import json
with open("prepared_data_nsp_main_2019.json", "w") as f:
    json.dump(result, f)

In [0]:
%ls -al --block-size=M

In [0]:
!cp /content/prepared_data_2018.json "/content/drive/My Drive/QA/Prepared data for tasks/prepared_data_nsp_main_2018.json"
